# LangChain process

In [1]:

from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from chromadb import Client as ChromaClient
from chromadb.config import Settings
from dotenv import load_dotenv
from voyageai import Client
from langchain.embeddings.base import Embeddings
from langchain.schema import Document
import logging
import os

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
VO_API_KEY = os.getenv("VO_API_KEY")

In [3]:

# Initialize VoyageAI client
vo = Client(api_key=VO_API_KEY)

In [4]:

# Define a custom embedding class
class CustomVoyageEmbeddings(Embeddings):
    def embed_query(self, query: str):
        try:
            embedding = vo.embed(query, model="voyage-3-large", input_type="document", output_dimension=1024).embeddings[0]
            logging.info(f"Generated embedding for query: {query}")
            return embedding
        except Exception as e:
            logging.error(f"Error generating query embedding: {e}")
            raise

    def embed_documents(self, texts: list[str]):
        try:
            embeddings = [
                vo.embed(text, model="voyage-3-large", input_type="document", output_dimension=1024).embeddings[0]
                for text in texts
            ]
            logging.info(f"Generated embeddings for {len(texts)} documents.")
            return embeddings
        except Exception as e:
            logging.error(f"Error generating document embeddings: {e}")
            raise

In [5]:

# Initialize custom embedding class
voyage_embeddings = CustomVoyageEmbeddings()

# Initialize Chroma client and store
from langchain_chroma import Chroma  # Updated import

In [6]:

def initialize_chroma(persist_directory="./chroma_data"):
    try:
        # Ensure the persistence directory exists
        if not os.path.exists(persist_directory):
            os.makedirs(persist_directory)
            logging.info(f"Created persistence directory: {persist_directory}")

        # Initialize Chroma vector store with persistence
        chroma_store = Chroma(
            collection_name="qa_embeddings_voyage-3-large_1024",
            embedding_function=voyage_embeddings,
            persist_directory=persist_directory,  # Enable persistence
        )
        logging.info(f"Chroma vector store initialized with persistence at: {persist_directory}")
        return chroma_store
    except Exception as e:
        logging.error(f"Error initializing Chroma vector store: {e}")
        raise


In [7]:

# Initialize Anthropic client
def initialize_anthropic():
    try:
        anthropic_llm = ChatAnthropic(
            model="claude-3-haiku-20240307",
            anthropic_api_key=ANTHROPIC_API_KEY
        )
        logging.info("Anthropic client initialized.")
        return anthropic_llm
    except Exception as e:
        logging.error(f"Error initializing Anthropic client: {e}")
        raise


In [8]:

# Define Retrieval-based QA chain
def initialize_qa_chain(chroma_store, anthropic_llm, n_neighbors=5):
    # Define a retriever with the specified number of neighbors
    retriever = chroma_store.as_retriever(search_kwargs={"k": n_neighbors})
    
    # Define a prompt template
    prompt_template = """
    You are an AI assistant designed to answer questions based on provided context. Your task is to analyze the given information and provide accurate, specific answers using only the data provided.

First, here's the context you'll be working with:

<context>
{context}
</context>

Now, here's the question you need to answer:

<question>
{question}
</question>

Please follow these steps to formulate your response:

1. Carefully read and analyze the provided context.
2. Examine the question and determine if the answer can be found in the context.
3. If you find the answer in the context, set 'found_answer' to 1. Otherwise, set it to 0.
4. Formulate your answer based solely on the information in the context. Do not use any external knowledge.
5. Construct your response in the specified JSON format.

Before providing your final answer, wrap your analysis in <analysis> tags. In your analysis:
- Write down relevant quotes from the context that might answer the question
- Analyze each quote for its relevance and how it might answer the question
- Consider if there are any gaps in the information provided
- Determine if the answer is clearly stated in the context
- Identify any relevant details or nuances in the context that relate to the question
- If you can't find an exact answer, consider if there's any information that partially addresses the question

Your final response should be in the following txt format:

```txt
found_answer: 0 or 1
```

Remember:
- Be very specific in your answer.
- Use only the information provided in the context.
- If you cannot find an answer in the context, clearly state this in your response.

Now, please proceed with your analysis and response.
    """
    
    qa_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # Define a custom chain to combine retrieved documents
    def combine_documents(docs):
        """Combine the content of retrieved documents."""
        return "\n\n".join([doc.page_content for doc in docs])

    class CustomRetriever:
        """Custom retriever to integrate document joining."""
        def __init__(self, retriever):
            self.retriever = retriever

        def get_relevant_documents(self, query):
            """Retrieve documents and combine them."""
            docs = self.retriever.get_relevant_documents(query)
            context = combine_documents(docs)
            return context, docs

    custom_retriever = CustomRetriever(retriever)

    # Define a custom QA chain
    class CustomRetrievalQA:
        """Custom Retrieval QA to integrate combined context."""
        def __init__(self, llm, retriever, prompt_template):
            self.llm = llm
            self.retriever = retriever
            self.prompt_template = prompt_template

        def __call__(self, inputs):
            query = inputs["query"]
            context, source_documents = self.retriever.get_relevant_documents(query)
            prompt = self.prompt_template.format(context=context, question=query)
            answer = self.llm(prompt)
            return {"result": answer, "source_documents": source_documents}

    qa_chain = CustomRetrievalQA(
        llm=anthropic_llm,
        retriever=custom_retriever,
        prompt_template=prompt_template
    )
    logging.info("Custom Retrieval-based QA chain initialized.")
    return qa_chain

In [24]:

import re 
# Main function to ask questions
def ask_question(qa_chain, query):
    logging.info(f"Asking question: {query}")
    result = qa_chain({"query": query})
    print("Answer:", result["result"].content)
    # print("\nRetrieved Documents:")
    # for doc in result["source_documents"]:
    #     print(f"Context: {doc.page_content}")
    #     print(f"Metadata: {doc.metadata}")

    # Regex to capture the value of found_answer
    pattern = r"</analysis>\s*found_answer:\s*(\d)"

    # Search for the pattern
    match = re.search(pattern, result["result"].content)

    if match:
        found_answer = int(match.group(1))
        return found_answer
    else:
        return 0

In [25]:

# Initialize components
chroma_store = initialize_chroma()
anthropic_llm = initialize_anthropic()
qa_chain = initialize_qa_chain(chroma_store, anthropic_llm, n_neighbors=10)

2025-01-20 19:16:14,237 - INFO - Chroma vector store initialized with persistence at: ./chroma_data
2025-01-20 19:16:14,237 - INFO - Anthropic client initialized.
2025-01-20 19:16:14,238 - INFO - Custom Retrieval-based QA chain initialized.


In [26]:

# Example query
query = 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'
result = ask_question(qa_chain, query)


2025-01-20 19:16:17,376 - INFO - Asking question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:16:17,694 - INFO - Generated embedding for query: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:16:19,119 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Answer: <analysis>
The provided context mentions that "the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."

This quote directly addresses the question of to whom the Virgin Mary allegedly appeared in 1858 in Lourdes, France. The context states that it was to Saint Bernadette Soubirous.
</analysis>

found_answer: 1
Saint Bernadette Soubirous


In [27]:
result

1

In [28]:
from datasets import load_dataset
# Load and filter dataset
dataset = load_dataset("squad", split="train[:1000]")
unique_contexts = set()
filtered_samples = []
for sample in dataset:
    context = sample["context"]
    if context not in unique_contexts:
        unique_contexts.add(context)
        filtered_samples.append(sample)


In [29]:
len(filtered_samples)

119

In [30]:
all_answers = []
for row in filtered_samples:
    print(row["question"])
    result = ask_question(qa_chain, query)
    all_answers.append(result)
    print(f"-"*40)

2025-01-20 19:16:30,681 - INFO - Asking question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


2025-01-20 19:16:30,937 - INFO - Generated embedding for query: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:16:32,455 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-01-20 19:16:32,459 - INFO - Asking question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


Answer: <analysis>
The context provided mentions the Grotto, "a Marian place of prayer and reflection" which is "a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858." This directly answers the question of to whom the Virgin Mary allegedly appeared in Lourdes, France in 1858 - Saint Bernadette Soubirous.
</analysis>

found_answer: 1

The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858.
----------------------------------------
When did the Scholastic Magazine of Notre dame begin publishing?


2025-01-20 19:16:32,778 - INFO - Generated embedding for query: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:16:34,172 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-01-20 19:16:34,176 - INFO - Asking question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


...

Answer: <analysis>
The context mentions that "The Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858." This quote directly answers the question, indicating that the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858.
</analysis>

found_answer: 1

The Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858.
----------------------------------------
Beyonce, during October 2014, partnered with whom to produce an outdoor line of clothing?


2025-01-20 19:20:03,122 - INFO - Generated embedding for query: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:20:04,602 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-01-20 19:20:04,607 - INFO - Asking question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


Answer: <analysis>
The context provided mentions that "The Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."

This quote directly answers the question, indicating that the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in Lourdes, France in 1858.
</analysis>

found_answer: 1
Saint Bernadette Soubirous
----------------------------------------
What national disaster caused Beyonce to create the Survivor Foundation?


2025-01-20 19:20:04,908 - INFO - Generated embedding for query: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
2025-01-20 19:20:06,744 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Answer: <analysis>
The context provides the following relevant information:
"The Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."
This quote directly states that the Virgin Mary appeared to Saint Bernadette Soubirous in the grotto at Lourdes, France in 1858.
</analysis>

found_answer: 1
```txt
Saint Bernadette Soubirous
```
----------------------------------------


In [31]:
all_answers

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [33]:
100*sum(all_answers)/len(all_answers)

97.47899159663865